## Dataset Overview

In [1]:
import pandas as pd, numpy as np, re, ast, math
from sklearn.preprocessing import MultiLabelBinarizer

df = pd.read_csv('dataset.csv')
df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/
3,Lightyear,"$71,101,257",5.2,32K,"['Animation', 'Action', 'Adventure']",While spending years attempting to return home...,"['galaxy', 'spaceship', 'robot', 'rocket', 'sp...",Angus MacLane,"['Jason Headley', 'Matthew Aldrich', 'Chris Ev...",Angus MacLane,-2022,/title/tt10298810/
4,Spiderhead,not-released,5.4,23K,"['Action', 'Crime', 'Drama']","In the near future, convicts are offered the c...","['discover', 'medical', 'test', 'reality', 'fi...",Joseph Kosinski,"['Rhett Reese', 'Paul Wernick', 'Chris Hemswor...",George Saunders,-2022,/title/tt9783600/


In [2]:
df.describe().T

,count,unique,top,freq
movie title,24402,23922,Rage,4
Run Time,24402,1556,not-released,8475
Rating,24402,91,no-rating,1740
User Rating,24402,1684,0,1740
Generes,24402,746,['Drama'],943
Overview,24158,23957,none,142
Plot Kyeword,24402,21546,[],1696
Director,24402,11604,See company contact information,142
Top 5 Casts,24402,24211,"['See producer', 'See preliminary cast']",142
Writer,24402,15562,See writer,142


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24402 entries, 0 to 24401
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie title   24402 non-null  object
 1   Run Time      24402 non-null  object
 2   Rating        24402 non-null  object
 3   User Rating   24402 non-null  object
 4   Generes       24402 non-null  object
 5   Overview      24158 non-null  object
 6   Plot Kyeword  24402 non-null  object
 7   Director      24402 non-null  object
 8   Top 5 Casts   24402 non-null  object
 9   Writer        24402 non-null  object
 10  year          23624 non-null  object
 11  path          24402 non-null  object
dtypes: object(12)
memory usage: 2.2+ MB


In [4]:
df.shape

(24402, 12)

In [5]:
df.isnull().sum()

movie title       0
Run Time          0
Rating            0
User Rating       0
Generes           0
Overview        244
Plot Kyeword      0
Director          0
Top 5 Casts       0
Writer            0
year            778
path              0
dtype: int64


# Dataset Cleaning and Feature Engineering Pipeline

The pipeline processes a movie dataset containing information such as titles, ratings, runtime, budget, genres, and more. It performs the following operations:

1. Standardize column names
2. Parse and clean various data types (runtime, budget, votes, year, etc.)
3. Extract features from existing columns
4. Create one-hot encodings for categorical data
5. Add derived metrics and flags
6. Remove duplicates

## Functions and Their Purpose

### 1. `standardise_columns(df)`

This function standardizes column names by:
- Stripping whitespace
- Converting to lowercase
- Replacing spaces with underscores (snake_case)
- Fixing specific misspellings:
  - "plot_kyeword" → "plot_keyword"
  - "generes" → "genres"

### 2. Helper Parser Functions

#### `parse_runtime_budget(raw)`
- Parses a string that might contain runtime or budget information
- Returns a tuple of (runtime_minutes, budget_amount)
- Handles various formats:
  - Hours and minutes format: "2 hours 27 minutes" → (147, NaN)
  - Budget format: "$15,000,000" → (NaN, 15000000.0)
  - Plain minutes: "105" → (105, NaN)
  - "not-released" → (NaN, NaN)

#### `parse_votes(raw)`
- Converts vote counts like "187K" or "1.2M" to integers
- Handles K (thousands) and M (millions) suffixes
- Examples:
  - "187K" → 187000
  - "1.2M" → 1200000

#### `parse_year(raw)`
- Extracts a 4-digit year from a string
- Validates that the year is between 1880 and 2025
- Returns NaN for invalid years

#### `parse_list(raw)`
- Converts string representations of lists into actual Python lists
- Handles both literal list syntax ("[item1, item2]") and comma-separated strings
- Strips whitespace from list items
- Returns an empty list if parsing fails

### 3. `cleanse_and_engineer(df_raw)`

This is the main pipeline function that:

#### A. Splits runtime and budget
- Applies `parse_runtime_budget()` to extract runtime minutes and budget values
- Creates separate columns "runtime_min" and "budget_usd"

#### B. Converts ratings and votes to numeric
- Converts "rating" to numeric format using pandas' `to_numeric`
- Applies `parse_votes()` to convert vote counts (like "187K") to integers

#### C. Processes year data
- Creates a clean "year_int" column using `parse_year()`
- Adds a "decade" column by integer division and multiplication by 10
  - Example: 1986 → 1980

#### D. Processes genres
- Converts the "genres" column to a list using `parse_list()`
- Uses `MultiLabelBinarizer` to create one-hot encoded columns for each genre
  - Examples: "genre_action", "genre_drama", "genre_sci_fi"

#### E. Adds additional features and flags
- Cleans the "overview" column by replacing "none" with NaN
- Creates a binary flag "has_keywords" (1 if the movie has plot keywords, 0 otherwise)
- Calculates a weighted popularity score "popularity_w" as rating × log10(votes + 1)
- Adds "is_top_rated" flag for movies with ratings ≥ 7.5
- Adds "is_recent" flag for movies from 2015 or later

#### F. Removes duplicates
- Sorts by "votes" (descending) to keep the most popular version
- Drops duplicates based on "movie_title" and "year_int", keeping the first occurrence

### Output

The function returns:
1. The cleaned and enhanced DataFrame
2. The fitted `MultiLabelBinarizer` instance used for genre encoding (useful for processing new data)

## Usage

```python
# Import and use the pipeline
df_raw = pd.read_csv('dataset.csv')  # Load your raw data
df = standardise_columns(df_raw)  # Standardize column names
clean_df, genre_encoder = cleanse_and_engineer(df)  # Run the full pipeline
```

### Example of Using the Fitted MultiLabelBinarizer on New Data

```python
# Have new data
new_dataset_df = pd.read_csv('new_dataset.csv')
new_dataset_df = standardise_columns(new_dataset_df)

# Parse the genres into lists
new_dataset_df["genre_list"] = new_dataset_df["genres"].apply(parse_list)

# Use the previously fitted mlb to transform - DON'T fit again!
new_genre_ohe = pd.DataFrame(
    mlb.transform(new_dataset_df["genre_list"]),
    columns=[f"genre_{g.lower().replace(' ', '_')}" for g in mlb.classes_],
    index=new_dataset_df.index
)

# Add to the new dataframe
new_dataset_df = pd.concat([new_dataset_df, new_genre_ohe], axis=1)
```

This approach ensures that:
1. If the new data has a genre like "Western" that wasn't in the original data, it's ignored
2. If the original data had "Musical" but none of the new movies are musicals, you still get a "genre_musical" column (all zeros)
3. The columns appear in the same order as they did in the original transformation

## Key Transformations Summary

- **Text standardization**: Clean column names, overview text
- **Numeric extraction**: Parse runtime, budget, votes, and ratings
- **Temporal features**: Extract year and decade
- **Categorical encoding**: One-hot encoding for genres
- **Feature engineering**: Weighted popularity, binary flags for quality and recency
- **Data integrity**: Deduplication based on title and year

In [6]:
def standardise_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    ▸ strip spaces, lower‑case, snake_case  
    ▸ fix common misspellings → plot_keyword, genres
    """
    df = df.copy()
    df.columns = (df.columns
                    .str.strip()
                    .str.lower()
                    .str.replace(" ", "_"))
    return df.rename(columns={"plot_kyeword": "plot_keyword",
                              "generes": "genres"})

df = standardise_columns(df)

In [7]:
def parse_runtime_budget(raw):
    if pd.isna(raw): return (np.nan, np.nan)
    s = str(raw).strip().lower()
    if s == "not-released": return (np.nan, np.nan)

    hh_mm = re.match(r'(?:(\d+)\s*hours?)?\s*(\d+)?\s*minutes?', s)
    if hh_mm and (hh_mm.group(1) or hh_mm.group(2)):
        hrs  = int(hh_mm.group(1) or 0)
        mins = int(hh_mm.group(2) or 0)
        return (hrs*60 + mins, np.nan)

    if s.isdigit():                                    # plain minutes
        return (int(s), np.nan)

    money = re.match(r'[\$€£]\s*([\d,]+(?:\.\d+)?)', s) # budget string
    if money:
        return (np.nan, float(money.group(1).replace(",", "")))

    return (np.nan, np.nan)

In [8]:
def parse_votes(raw):
    if pd.isna(raw): return np.nan
    s = str(raw).strip().lower()
    mult = 1_000 if s.endswith("k") else 1_000_000 if s.endswith("m") else 1
    if mult != 1: s = s[:-1]          # drop k/m suffix
    try: return int(float(s.replace(",", "")) * mult)
    except ValueError: return np.nan

In [9]:
def parse_year(raw):
    if pd.isna(raw): return np.nan
    m = re.search(r'(\d{4})', str(raw))
    if m:
        y = int(m.group(1))
        return y if 1880 <= y <= 2025 else np.nan
    return np.nan

In [10]:
def parse_list(raw):
    if pd.isna(raw): return []
    if isinstance(raw, list): return raw
    try:
        lst = ast.literal_eval(raw)
        if isinstance(lst, list):
            return [x.strip() for x in lst if x.strip()]
    except (ValueError, SyntaxError):
        pass
    return [x.strip() for x in str(raw).split(",") if x.strip()]

In [11]:
def cleanse_and_engineer(df_raw: pd.DataFrame):
    df = df_raw.copy()

    # split runtime/budget
    rt_bud = df["run_time"].apply(parse_runtime_budget)
    df["runtime_min"] = rt_bud.apply(lambda x: x[0])
    df["budget_usd"]  = rt_bud.apply(lambda x: x[1])

    # numeric ratings / votes
    df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
    df["votes"]  = df["user_rating"].apply(parse_votes)

    # year & decade
    df["year_int"] = df["year"].apply(parse_year)
    df["decade"]   = (df["year_int"] // 10) * 10

    # genres list → one‑hot
    df["genre_list"] = df["genres"].apply(parse_list)
    mlb = MultiLabelBinarizer()
    genre_ohe = pd.DataFrame(
        mlb.fit_transform(df["genre_list"]),
        columns=[f"genre_{g.lower().replace(' ', '_')}" for g in mlb.classes_],
        index=df.index
    )
    df = pd.concat([df, genre_ohe], axis=1)

    # misc text fix & boolean flags
    df["overview"]      = df["overview"].replace({"none": np.nan})
    df["has_keywords"]  = df["plot_keyword"].apply(
                              lambda x: int(bool(parse_list(x))))
    df["popularity_w"]  = df["rating"] * np.log10(df["votes"] + 1)
    df["is_top_rated"]  = (df["rating"] >= 7.5).astype(int)
    df["is_recent"]     = (df["year_int"] >= 2015).astype(int)

    # dedupe on (title, year) keeping highest‑vote record
    df = (df.sort_values("votes", ascending=False)
            .drop_duplicates(subset=["movie_title", "year_int"],
                             keep="first"))

    return df, mlb          # return the clean frame + fitted encoder

In [12]:
clean_df, genre_encoder = cleanse_and_engineer(df)

print("After cleansing head:\n",
      clean_df[["movie_title", "runtime_min", "budget_usd",
                 "rating", "votes", "year_int",
                 "genre_action", "genre_drama", "is_top_rated"]].head())
print("\nFinal shape:", clean_df.shape)
print("\n", clean_df.columns.tolist())

After cleansing head:
                    movie_title  runtime_min   budget_usd  rating    votes  \
8021  The Shawshank Redemption          NaN   25000000.0     9.3  2600000   
51             The Dark Knight          NaN  185000000.0     9.0  2600000   
57                   Inception          NaN  160000000.0     8.8  2300000   
8039              Forrest Gump          NaN   55000000.0     8.8  2000000   
8042              Pulp Fiction          NaN    8000000.0     8.9  2000000   

      year_int  genre_action  genre_drama  is_top_rated  
8021    1994.0             0            1             1  
51      2008.0             1            1             1  
57      2010.0             1            0             1  
8039    1994.0             0            1             1  
8042    1994.0             0            1             1  

Final shape: (23922, 46)

 ['movie_title', 'run_time', 'rating', 'user_rating', 'genres', 'overview', 'plot_keyword', 'director', 'top_5_casts', 'writer', 'year', 'p